# Simulación de eficiencia de fotodetectores basados en semiconductores

Autores:
- Santiago Vélez Arboleda
- César Augusto Montoya Ocampo

## Librerías

In [8]:
import numpy as np
import matplotlib.pyplot as plt
import pint

# Importar el UnitRegistry de constants.py para que todas las unidades sean consistentes
from constants import u 

# Importar las clases que hemos definido
from semiconductor import Semiconductor
from photodetector import Photodetector

# Configurar Matplotlib para gráficos más bonitos
plt.rcParams.update({
    'font.size': 12,
    'axes.labelsize': 14,
    'axes.titlesize': 16,
    'xtick.labelsize': 12,
    'ytick.labelsize': 12,
    'legend.fontsize': 12,
    'figure.figsize': (10, 6),
    'lines.linewidth': 2,
    'grid.linestyle': '--',
    'grid.linewidth': 0.5
})

print("Librerías y módulos importados exitosamente.")

Librerías y módulos importados exitosamente.


In [9]:
# Celda 2

# 1. Definir los datos de un material semiconductor (ej: Silicio)
# Estos son datos simplificados para un ejemplo. En un caso real, serían más extensos y precisos.
silicon_data = {
    "name": "Silicio (Si)",
    "band_gap_0K": 1.17 * u.eV, # Band gap a 0K
    "varshni_alpha": 4.73e-4 * u.eV / u.kelvin**2,
    "varshni_beta": 636 * u.kelvin,
    "electron_effective_mass": 1.08, # Factor de la masa libre del electrón
    "hole_effective_mass": 0.56,   # Factor de la masa libre del electrón
    "relative_permittivity": 11.7, # Constante dieléctrica relativa
    "high_freq_permittivity": 11.7, # Similar a la relativa para Si
    "sound_velocity": 9000 * u.meter / u.second, # Valor aproximado
    "deformation_potential": 5.0 * u.eV, # Valor aproximado
    "density": 2329 * u.kg / u.meter**3,
    "elastic_constant": 1.66e11 * u.pascal, # Para dispersión inter-valle
    "num_equivalent_valleys": 6, # Para el silicio
    "LO_phonon_energy": 0.063 * u.eV, # Aproximado para Si, necesario si es polar
    "is_polar": False, # El Si es no polar
    "electron_lifetime": 10e-6 * u.second, # Ejemplo: 10 microsegundos
    "hole_lifetime": 10e-6 * u.second,     # Ejemplo: 10 microsegundos
    "absorption_coefficient_data": { # Datos de absorción (ejemplo simplificado)
        # Longitud de onda (nm): Coeficiente de absorción (cm^-1)
        300: 1.0e5 * u.cm**-1, # <-- ¡CORRECCIÓN AQUÍ!
        400: 5.0e4 * u.cm**-1, # <-- ¡CORRECCIÓN AQUÍ!
        500: 1.0e4 * u.cm**-1, # <-- ¡CORRECCIÓN AQUÍ!
        600: 3.0e3 * u.cm**-1, # <-- ¡CORRECCIÓN AQUÍ!
        700: 1.0e3 * u.cm**-1, # <-- ¡CORRECCIÓN AQUÍ!
        800: 3.0e2 * u.cm**-1, # <-- ¡CORRECCIÓN AQUÍ!
        900: 8.0e1 * u.cm**-1, # <-- ¡CORRECCIÓN AQUÍ!
        1000: 1.0e1 * u.cm**-1, # <-- ¡CORRECCIÓN AQUÍ!
        1100: 1.0e0 * u.cm**-1, # <-- ¡CORRECCIÓN AQUÍ!
        1200: 0.1 * u.cm**-1    # <-- ¡CORRECCIÓN AQUÍ!
    }
}

# 2. Crear una instancia del Semiconductor
silicon = Semiconductor(material_data=silicon_data)
print(f"Material semiconductor instanciado: {silicon.name}")

# 3. Definir los parámetros de un fotodetector simple
# Espesor de la capa activa: 10 micrómetros
detector_thickness = 10 * u.micrometer
# Reflectividad superficial: 30% (ejemplo)
detector_reflectivity = 0.30

# 4. Crear una instancia del Fotodetector
simple_photodetector = Photodetector(
    semiconductor=silicon,
    thickness=detector_thickness,
    surface_reflectivity=detector_reflectivity
)

print(f"\nFotodetector instanciado con material: {simple_photodetector.semiconductor.name}")
print(f"Espesor del fotodetector: {simple_photodetector.thickness.to(u.micrometer):.0f}")
print(f"Reflectividad superficial: {simple_photodetector.surface_reflectivity:.2f}")

print("\n¡Listo para calcular la eficiencia cuántica y la responsividad!")

Material semiconductor instanciado: Silicio (Si)

Fotodetector instanciado con material: Silicio (Si)
Espesor del fotodetector: 10 micrometer
Reflectividad superficial: 0.30

¡Listo para calcular la eficiencia cuántica y la responsividad!


In [ ]:
# Celda 3

# Definir el rango de longitudes de onda para el cálculo
# Vamos a ir desde 300 nm (UV) hasta 1200 nm (NIR cercano)
# El Silicio es más sensible en el rango visible a infrarrojo cercano.
wavelengths = np.linspace(300, 1200, 200) # 200 puntos entre 300 y 1200 nm
wavelengths = [wavelength * u.nm for wavelength in wavelengths]  # Asegurar que son objetos pint

# Definir una temperatura de operación (ej: temperatura ambiente)
operation_temperature = 300 * u.kelvin

# Listas para almacenar los resultados
quantum_efficiencies = []
responsivities = []

# Calcular la eficiencia cuántica y la responsividad para cada longitud de onda
for wl in wavelengths:
    qe = simple_photodetector.get_quantum_efficiency(wl, operation_temperature)
    resp = simple_photodetector.get_responsivity(wl, operation_temperature)

    quantum_efficiencies.append(qe.to(u.dimensionless).magnitude) # Convertir a valor numérico adimensional
    responsivities.append(resp.to(u.ampere/u.watt).magnitude)    # Convertir a valor numérico en A/W

# Convertir listas a arrays de numpy para facilitar el graficado
quantum_efficiencies = np.array(quantum_efficiencies)
responsivities = np.array(responsivities)

# --- Graficar la Eficiencia Cuántica ---
plt.figure(figsize=(10, 6))
plt.plot(wavelengths.to(u.nm).magnitude, quantum_efficiencies, label='Eficiencia Cuántica')
plt.xlabel('Longitud de Onda (nm)')
plt.ylabel('Eficiencia Cuántica ($\eta$)')
plt.title(f'Eficiencia Cuántica del Fotodetector de {simple_photodetector.semiconductor.name} (Espesor: {simple_photodetector.thickness.to(u.micrometer):.0f})')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

# --- Graficar la Responsividad ---i
plt.figure(figsize=(10, 6))
plt.plot(wavelengths.to(u.nm).magnitude, responsivities, label='Responsividad')
plt.xlabel('Longitud de Onda (nm)')
plt.ylabel('Responsividad (A/W)')
plt.title(f'Responsividad del Fotodetector de {simple_photodetector.semiconductor.name} (Espesor: {simple_photodetector.thickness.to(u.micrometer):.0f})')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

print("\nCálculos y gráficos completados.")

<class 'pint.Quantity'>


<>:33: SyntaxWarning: invalid escape sequence '\e'
<>:33: SyntaxWarning: invalid escape sequence '\e'
/tmp/ipykernel_63368/1124135707.py:33: SyntaxWarning: invalid escape sequence '\e'
  plt.ylabel('Eficiencia Cuántica ($\eta$)')
/tmp/ipykernel_63368/1124135707.py:33: SyntaxWarning: invalid escape sequence '\e'
  plt.ylabel('Eficiencia Cuántica ($\eta$)')


AttributeError: 'numpy.float64' object has no attribute 'to'